## Pathway Ranking (work in progress)

In [1]:
# import a variety of packages
import requests
import yaml
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
!pip2 install --quiet pypathway # not instandard jupyter container so add manually
from pypathway import *
import pymongo

In [62]:
# Connect to the database
db = pymongo.MongoClient("mongo").MedBook

In [7]:
# Find all jobs from a given sample name
jobs = db.jobs.find({"args.sample_label": "ckcc/TH03_0118_S01_RNASeq"})
for job in jobs:
    print job["_id"],":", job["args"]["sample_label"],"against", job["args"]["sample_group_name"]

GsaN5gNNrusKL8DtX : ckcc/TH03_0118_S01_RNASeq against --[TC2] acute myeloid leukemia
KFN3WuD6vRdun3ih3 : ckcc/TH03_0118_S01_RNASeq against --[TC2] AML+ALAL+ALL
hzTfpy7kXzftFMJxe : ckcc/TH03_0118_S01_RNASeq against Compendium 2 final samples 2016/10/24
zmkf3FzT9ajMPeWLC : ckcc/TH03_0118_S01_RNASeq against AML


In [69]:
# Get the results of a specific job
job = db.jobs.find_one({"_id": "hzTfpy7kXzftFMJxe"})
up = pd.DataFrame(job["output"]["up_genes"])
down = pd.DataFrame(job["output"]["down_genes"])
top5percent = pd.DataFrame(job["output"]["top5percent_genes"])
print "Outlier analysis for", job["args"]["sample_label"],"against", job["args"]["sample_group_name"]

Outlier analysis for ckcc/TH03_0118_S01_RNASeq against Compendium 2 final samples 2016/10/24


In [70]:
# Search for up regulated drug interactions - see http://dgidb.genome.wustl.edu/api
endpoint = "http://dgidb.genome.wustl.edu/api/v1/interactions.json"
query = "?genes={}&interaction_type=inhibitor".format(",".join(up["gene_label"].values))
matches = requests.get(endpoint + query).json()
druggable_genes = {m["geneName"]: list(set([i["drugName"] 
                                            for i in m["interactions"]])) for m in matches["matchedTerms"]}
print "Up regulated genes with drug targets:"
print yaml.safe_dump(druggable_genes, default_flow_style=False)

Up regulated genes with drug targets:
ADA:
- PENTOSTATIN
- 1-((1R,2S)-1-{2-[2-(4-CHLOROPHENYL)-1,3-BENZOXAZOL-7-YL]ETHYL}-2-HYDROXYPROPYL)-1H-IMIDAZOLE-4-CARBOXAMIDE
- 1-((1R)-1-(HYDROXYMETHYL)-3-{6-[(3-PHENYLPROPANOYL)AMINO]-1H-INDOL-1-YL}PROPYL)-1H-IMIDAZOLE-4-CARBOXAMIDE
- 1-{(1R,2S)-2-HYDROXY-1-[2-(2-NAPHTHYLOXY)ETHYL]PROPYL}-1H-IMIDAZONE-4-CARBOXAMIDE
- FLUDARABINE
- CLADRIBINE
- EHNA
- DIPYRIDAMOLE
- 6-HYDROXY-7,8-DIHYDRO PURINE NUCLEOSIDE
- 6-HYDROXY-1,6-DIHYDRO PURINE NUCLEOSIDE
APAF1:
- ADENOSINE TRIPHOSPHATE
BTK:
- AVL-292
- INOSITOL 1,3,4,5-TETRAKISPHOSPHATE
- IBRUTINIB
CFD:
- 3,4-DICHLOROISOCOUMARIN
- ISATOIC ANHYDRIDE
CSF3R:
- RUXOLITINIB
CTSG:
- BIS-NAPTHYL BETA-KETOPHOSPHONIC ACID
- DERMOLASTIN
- 2-[3-({METHYL[1-(2-NAPHTHOYL)PIPERIDIN-4-YL]AMINO}CARBONYL)-2-NAPHTHYL]-1-(1-NAPHTHYL)-2-OXOETHYLPHOSPHONIC
  ACID
DNTT:
- CORDYCEPIN
FLT3:
- LESTAURTINIB
- QUIZARTINIB
- GTP-14564
- SORAFENIB
- FLT-3 INHIBITOR II
- CRENOLANIB
- 4SC-202
- RGB-286638
- CEDIRANIB
- CLOFARABINE
- S

In [71]:
# Lookup all pathways by gene
from collections import defaultdict
pathways = defaultdict(list)
for gene in druggable_genes.keys():
    pathways[gene].extend(PublicDatabase.search_kegg(gene, organism="hsa"))
    pathways[gene].extend(PublicDatabase.search_reactome(gene))
    pathways[gene].extend(PublicDatabase.search_wp(gene))

In [47]:
up_genes_set = set([name.upper() for name in top5percent["gene_label"].values])

In [54]:
[v["diagram"].title for k, v in candidates.iteritems() if len(v["shared"]) > 0]

[u'Selenocompound metabolism',
 u'Glycosaminoglycan degradation',
 u'Valine, leucine and isoleucine degradation',
 u'Lysine degradation',
 u'RNA degradation',
 u'Caprolactam degradation',
 u'Fatty acid degradation',
 u'Two-component system']

In [49]:
candidates = {}
up_genes_set = set([name.upper() for name in top5percent["gene_label"].values])
for pathway in pathways:
    if pathway.id in candidates:
        print "duplicate", pathway.id
        continue
    diagram = pathway.load()
    shared = set([e.display_name.upper() if e.display_name else None for e in diagram.entities]).intersection(up_genes_set)
    candidates[pathway.id] = {"diagram": diagram, "shared": shared}

duplicate WP511
duplicate WP314
duplicate WP89
duplicate WP1019
duplicate WP571
duplicate WP902
duplicate WP788
duplicate WP1138
duplicate WP1824
duplicate WP1396
duplicate WP1825
duplicate WP1824
duplicate WP3174
duplicate WP2849
duplicate WP3504
duplicate WP2841
duplicate WP3626
duplicate WP1826
duplicate WP1885
duplicate WP1087
duplicate WP744
duplicate WP151
duplicate WP44
duplicate WP849
duplicate WP1794
duplicate WP2309
duplicate WP1794
duplicate WP1829
duplicate 01220
duplicate WP2806
duplicate WP3504
duplicate WP147
duplicate WP1341
duplicate WP774
duplicate WP407
duplicate WP886
duplicate WP1121
duplicate WP127
duplicate WP908
duplicate WP285
duplicate WP1354
duplicate WP274
duplicate WP1144
duplicate WP794
duplicate WP1315
duplicate WP1087
duplicate WP304
duplicate WP1004
duplicate WP744
duplicate WP151
duplicate WP44
duplicate WP849
duplicate WP3885
duplicate WP1025
duplicate WP3884
duplicate WP23
duplicate WP1449
duplicate WP3132
duplicate WP2746


In [29]:
# Search for the kegg pathway with a druggable gene in it and number of up regulated genes in it
up_genes_set = set(up["gene_label"].values)
top_pathway = None
top_pathway_count = 0
for gene in druggable_genes.keys():
    print "Gene:", gene
    for resource in PublicDatabase.search_kegg(gene, organism="hsa"):
        pathway = resource.load()
        shared = set([e.display_name for e in pathway.entities]).intersection(up_genes_set)
        print "Found", len(shared), "genes"
        print shared
        if len(shared) > top_pathway_count:
            print "New top pathway"
            top_pathway_count = len(shared)
            top_pathway = pathway

Gene: APAF1
Gene: ITGA4
Gene: P2RX1
Gene: MAPKAPK3
Gene: FLT3
Gene: DNTT
Gene: S1PR4
Gene: PIK3R6
Gene: SUCNR1
Gene: RNASE2
Gene: PIM1
Gene: CTSG
Gene: SLC25A20
Gene: PADI4
Gene: CSF3R
Gene: ADA
Found 4 genes
set([u'CPT1A', u'HADHA', u'ACADVL', u'ACADM'])
New top pathway
Found 0 genes
set([])
Found 3 genes
set([u'IVD', u'ACADM', u'DLD'])
Found 3 genes
set([u'DLST', u'OGDH', u'PLOD1'])
Found 7 genes
set([u'MAN2B2', u'MANBA', u'GBA', u'HEXA', u'GLB1', u'MAN2C1', u'ENGASE'])
New top pathway
Found 5 genes
set([u'HEXA', u'GNS', u'HGSNAT', u'GLB1', u'GUSB'])
Found 9 genes
set([u'PAN3', u'PAN2', u'EDC4', u'LSM4', u'DDX6', u'XRN2', u'CNOT3', u'CNOT2', u'CNOT1'])
New top pathway
Gene: PTPN6
Gene: SELL
Gene: MPO
Found 1 genes
set([u'MARS'])
Gene: CFD
Gene: PRTN3
Gene: BTK


In [37]:
top_pathway.title

u'RNA degradation'

In [35]:
top_pathway.draw()

In [5]:
# get a list of id
id_list = [x.id for x in path.genes]
# generate a visualize option whose background color in default is red
vo = Option(default=[Prop(bg_color= "red")])
# set this visualize option to all gene in pathway
path.integrate(id_list, [vo for _ in id_list])

In [22]:
path.genes

[class: entry, name: hsa:4609, type: gene, id: 1, KO: [u'K04377'],
 class: entry, name: hsa:10000 hsa:207 hsa:208, type: gene, id: 2, KO: [u'K04456'],
 class: entry, name: hsa:23533 hsa:5290 hsa:5291 hsa:5293 hsa:5294 hsa:5295 hsa:5296 hsa:8503, type: gene, id: 3, KO: [u'K00922', u'K02649'],
 class: entry, name: hsa:6654 hsa:6655, type: gene, id: 4, KO: [u'K03099'],
 class: entry, name: hsa:2885, type: gene, id: 5, KO: [u'K04364'],
 class: entry, name: hsa:1154, type: gene, id: 6, KO: [u'K04701'],
 class: entry, name: hsa:596, type: gene, id: 7, KO: [u'K02161'],
 class: entry, name: hsa:122809 hsa:30837 hsa:8651 hsa:8835 hsa:9021 hsa:9306 hsa:9655, type: gene, id: 8, KO: [u'K04694', u'K04695', u'K04696', u'K04697', u'K04698', u'K04699'],
 class: entry, name: hsa:1387 hsa:2033, type: gene, id: 9, KO: [u'K04498'],
 class: entry, name: hsa:10379, type: gene, id: 10, KO: [u'K04693'],
 class: entry, name: hsa:6772 hsa:6773 hsa:6774 hsa:6775 hsa:6776 hsa:6777 hsa:6778, type: gene, id: 11, KO

In [7]:
path.draw()

In [48]:
# query pathway
# get a list of id
id_list = [x.id for x in path.genes]
# generate a visualize option whose background color in default is red
vo = Option(default=[Prop(bg_color= "red")])
# set this visualize option to all gene in pathway
path.integrate(id_list, [vo for _ in id_list])





# # now lets change all node's color to red
# vo = Option(default=[Prop(bg_color="red")])
# path.integrate([x.props["GraphId"] for x in path.nodes], [vo for _ in path.entities])
# path.draw()
# Out[3]:


In [51]:
path.draw()